## Overview

This notebook demonstrates how to use Monet to load and save gene expression data from/to disk.

Monet represents gene expression data as gene-by-cell matrices stored in `ExpMatrix` objects. The `ExpMatrix` class inherits from the pandas `DataFrame` and can be used in identical fashion. However, it provides a few additional features. For example, for aesthetic reasons, you can refer to the `index` (containing the gene names) using the `.gene` attribute, and you can refer to the `columns` (containing the cell names) using the `.cells` attribute.

### Setting up the notebook

In [9]:
# change notebook width and font
from IPython.core.display import HTML, display
display(HTML("""<style>
    /* source: http://stackoverflow.com/a/24207353 */
    /* .container { width:95% !important; }    */
    div.prompt, div.CodeMirror pre, div.output_area pre { font-family:'Hack', monospace; font-size: 10.5pt; }
    </style>"""))

from monet import util
_LOGGER = util.configure_logger()

## Loading data

The code demonstrates the various data formats supported by Monet for loading scRNA-Seq expression data. It also provides a comparison of the time required to load a PBMC dataset containing ~10,000 cells, depending on whether it's loaded from a Monet `.npz` file, or from a tab-delimited text file.

### Loading data from Monet binary files (.npz)

This is the default data format used by Monet to store data on disk. It uses NumPy's compressed binary `.npz` format, which allows scRNA-Seq data to be stored in a space-efficient manner, and also makes the data very fast to load. 

In [3]:
import time
import gc

from monet import ExpMatrix

expression_file = 'data/v3_human_pbmc_10k_expression.npz'

t0 = time.time()

# loading the expression matrix
matrix = ExpMatrix.load_npz(expression_file)

t1 = time.time()
_LOGGER.info('Loading data in binary .npz format took %.1f s.', t1-t0)

# free up memory
del matrix; gc.collect()

[2020-06-15 20:25:02] (monet.core.exp_matrix) INFO: Loaded expression matrix with 10681 cells and 16319 genes -- .npz format, 36.7 MB (hash: f9d7fac20f4de6184ff55388c267699a).
[2020-06-15 20:25:02] (root) INFO: Loading data in binary .npz format took 3.3 s.


78

### Loading data from tab-delimited plain-text files (.tsv.gz)

*Note: Both comma-delimited (csv) and tab-delimited (tsv) text files are supported using the same function.*

These are just a gzip'ed tab-delimited plain-text files that can also be read and written using pandas' `read_csv()` and `to_csv()` functions, respectively. It's also a very space-efficient way of storing scRNA-Seq data, but because it's not a binary format, it takes a lot longer to load. (Parsing text is slow!)

In [5]:
import time
import gc

from monet import ExpMatrix

expression_file = 'data/v3_human_pbmc_10k_expression.tsv.gz'

t0 = time.time()

# load the expression matrix
matrix = ExpMatrix.load_tsv(expression_file)
### Note: you can use ExpMatrix.load_tsv(..., sep=',') to load comma-delimited data

t1 = time.time()
_LOGGER.info('Loading data in tab-delimited .tsv.gz format took %.1f s.', t1-t0)

# free up memory
del matrix; gc.collect()

[2020-06-15 21:51:42] (monet.core.exp_matrix) INFO: Loaded expression matrix with 10681 cells and 16319 genes -- plain-text format, 21.0 MB (hash: f9d7fac20f4de6184ff55388c267699a).
[2020-06-15 21:51:42] (root) INFO: Loading data in tab-delimited .tsv.gz format took 29.0 s.


8

### Loading data produced by CellRanger (.tar.gz)

*Note: If you want to run the code in this section, you need to download [this file](http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz) from the 10x Genomics website.*

*Note: A computer with at least 8GB of RAM is recommended to run the code in this section.*

The PBMC data loaded in the previous examples was obtained by preprocessing a publicly available dataset called [10k PBMCs from a Healthy Donor (v3 chemistry)](https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_10k_v3). In this case, the expression quantification was done using the CellRanger software, which produces various output files. The most straightforward way of getting the expression data for this dataset (and other datasets provided by 10X Genomics) is to download the file called "[Feature / cell matrix (filtered)](http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz)". Here, CellRanger uses a plain-text format for storing sparse matrices called [MatrixMarket](https://math.nist.gov/MatrixMarket/formats.html) to store scRNA-Seq expression data. The names of the "features" (genes) and cell names are stored in separate text files. CellRanger 3.0.0 then combines all three files (expression matrix, gene names, cell names) into a compressed tar archive, where they are stored in a subdirectory called "filtered_feature_bc_matrix".

Below is an example of how to load such a dataset. Note that this dataset contains more than three times as many genes than the preprocessed versions above, which were filtered to only include protein-coding genes. Therefore the expression matrix is more than three times larger, and the load time is not directly comparable.

In [12]:
import time
import gc

from monet import ExpMatrix

# adjust path to where you downloaded the file
expression_file = '~/Dropbox/work/data/10x_genomics/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz'

t0 = time.time()

# load the expression matrix
matrix = ExpMatrix.load_10x_v3(expression_file, prefix='filtered_feature_bc_matrix/')

t1 = time.time()
_LOGGER.info('Loading data in sparse CellRanger format took %.1f s.', t1-t0)

# free up memory
del matrix; gc.collect()

[2020-06-15 22:34:31] (monet.core.exp_matrix) INFO: Loaded expression matrix with 11769 cells and 33538 genes -- CellRanger v3 sparse format, 94.3 MB (hash: 5ced0115a8e3bda1af1909ce503b43e2).
[2020-06-15 22:34:31] (root) INFO: Loading data in sparse CellRanger format took 46.9 s.


111

## Saving data

The code below demonstrates how to use Monet to save data in either Monet's compressed binary `.npz` format, or in in a standard compressed, tab-delimited plain-text `.tsv.gz` format. (The same formats as shown above under "Loading data".) The outupt shows that that saving data in Monet's binary `.npz` format is much faster compared to the plain-text `.tsv.gz` format.

In [14]:
import time
import gc

from monet import ExpMatrix

# load data
expression_file = 'data/v3_human_pbmc_10k_expression.npz'
matrix = ExpMatrix.load_npz(expression_file)

t0 = time.time()

# save data to compressed binary format
output_file = 'output/v3_human_pbmc_10k_expression.npz'
matrix.save_npz(output_file)

t1 = time.time()
_LOGGER.info('Saving data to binary .npz format took %.1f s.', t1-t0)

t0 = time.time()

# save data to compressed tab-delimited plain-text file
output_file = 'output/v3_human_pbmc_10k_expression.tsv.gz'
matrix.save_tsv(output_file)
### note: you can switch to comma-delimited format (csv) using matrix.save_tsv(..., sep=',')

t1 = time.time()
_LOGGER.info('Saving data to plain-text .tsv.gz format took %.1f s.', t1-t0)

# free up memory
del matrix; gc.collect()

[2020-06-15 22:51:39] (monet.core.exp_matrix) INFO: Loaded expression matrix with 10681 cells and 16319 genes -- .npz format, 36.7 MB (hash: f9d7fac20f4de6184ff55388c267699a).
[2020-06-15 22:51:52] (monet.core.exp_matrix) INFO: Saved expression matrix with 10681 cells and 16319 genes -- .npz format, 36.7 MB (hash: f9d7fac20f4de6184ff55388c267699a).
[2020-06-15 22:51:52] (root) INFO: Saving data to binary .npz format took 13.5 s.
[2020-06-15 22:54:25] (monet.core.exp_matrix) INFO: Saved expression matrix with 10681 cells and 16319 genes -- plain-text format, 21.0 MB (hash: f9d7fac20f4de6184ff55388c267699a).
[2020-06-15 22:54:25] (root) INFO: Saving data to plain-text .tsv.gz format took 153.0 s.


76